# Example on running an Experiment with mocked LabOne Q result data

In [ ]:
from laboneq.simple import *
from laboneq_library.experiments import qubit_calibration_experiments as qb_calib_exp
from laboneq_library import loading_helpers as load_hlp
from laboneq_library import calibration_helpers as calib_hlp

from laboneq_library.utils.debugging import mock_acquired_results, get_acquired_results_from_results_json

In [ ]:
from laboneq.dsl.device import create_connection
from laboneq.dsl.device.instruments import HDAWG, SHFQC, PQSC
measurement_setup = DeviceSetup("TopFeedlineT6C")
measurement_setup.add_dataserver(host="localhost", port="8004")
measurement_setup.add_instruments(
    HDAWG(uid="device_hdawg", address="dev8793"),
    SHFQC(uid="device_shfqc", address="dev12250"),
    PQSC(uid="device_pqsc", address="dev10091"),
)
for i, qbn in enumerate(['qb1', 'qb2', 'qb3']):
    measurement_setup.add_connections(
        "device_hdawg",
        create_connection(to_signal=f"{qbn}/flux_line", ports=f"SIGOUTS/{i+1}"),
    )

    measurement_setup.add_connections(
        "device_shfqc",
        create_connection(to_signal=f"{qbn}/drive_line", ports=f"SGCHANNELS/{str(i)}/OUTPUT"),
        create_connection(to_signal=f"{qbn}/drive_line_ef", ports=f"SGCHANNELS/{str(i)}/OUTPUT"),
        create_connection(to_signal=f"{qbn}/measure_line", ports="QACHANNELS/0/OUTPUT"),
        create_connection(to_signal=f"{qbn}/acquire_line", ports="QACHANNELS/0/INPUT"),
    )
measurement_setup.add_connections(
    "device_pqsc",
    create_connection(to_instrument="device_hdawg", ports="ZSYNCS/2"),
    create_connection(to_instrument="device_shfqc", ports="ZSYNCS/0"),
)

In [ ]:
qubit_parameters = load_hlp.load_qubit_parameters_from_json(
    full_filepath='qubit_parameters.json')
qb1, qb2, qb3 = calib_hlp.create_qubits_from_parameters(qubit_parameters, measurement_setup)
qubits = [qb1, qb2, qb3]

In [ ]:
session = Session(device_setup=measurement_setup)
session.connect(do_emulation=True, reset_devices=False)

In [ ]:
# Previous experiment results for Amplitude Rabi ef without active reset, 3 qubits
exp_results_json = "experiment_results.json"

for qubits_to_measure in [
    [qb1, qb3],
    [qb2]
]:
    for transition_to_calibrate in [
        'ge',
    ]:
        for preparation_type in [
            "wait",
        ]:
            qubit_temporary_values = []
            sweep_parameters_dict = {}
            for qubit in qubits_to_measure:
                qubit_temporary_values += [
                    (qubit, "reset_delay_length", 200e-6),
                ]

                pi_amp = qubit.parameters.drive_parameters_ef['amplitude_pi'] if transition_to_calibrate == "ef" else \
                    qubit.parameters.drive_parameters_ge['amplitude_pi']
                swp_end = pi_amp + 0.1
                sweep_parameters_dict[qubit.uid] = [
                    LinearSweepParameter(f'amps_{qubit.uid}', 0, swp_end, 21, 'Amplitude Scaling')]

            acquisition_metainfo = dict(count=2 ** 0)
            experiment_metainfo = dict(cal_states=transition_to_calibrate,
                                       transition_to_calibrate=transition_to_calibrate,
                                       preparation_type=preparation_type
                                      )
            analysis_metainfo = dict(show_figures=True, do_fitting=True)
            with mock_acquired_results(session, get_acquired_results_from_results_json(exp_results_json)) as session:
                exp = qb_calib_exp.AmplitudeRabi(qubits_to_measure, session, measurement_setup,
                                                experiment_metainfo=experiment_metainfo,
                                                acquisition_metainfo=acquisition_metainfo,
                                                analysis_metainfo=analysis_metainfo,
                                                sweep_parameters_dict=sweep_parameters_dict,
                                                qubit_temporary_values=qubit_temporary_values,
                                                do_analysis=True, update=False,
                                                data_directory="data", save=False, run=True)